<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Parameters Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>()
                       .ResetType<SingleDataNodeParameter>().ResetType<InterDataNodeParameter>());

# Setting Constants

In [0]:
var EnableScenarioBackup = EnableScenario;

In [0]:
EnableScenario = true;

In [0]:
var ws = Workspace.CreateNew();
ws.InitializeFrom(DataSource);

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioYieldCurve = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "YCUP1.0pct", ImportFormats.Cashflow);
var argsScenarioTransactionalData = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "SRUP1.0pct", ImportFormats.Cashflow);

In [0]:
var initStorage = new ImportStorage(argsBestEstimate, DataSource, ws);
await initStorage.InitializeAsync();

In [0]:
var actualEstimateTypes = initStorage.EstimateTypesByImportFormat[ImportFormats.Actual];
var cashflowEstimateTypes = initStorage.EstimateTypesByImportFormat[ImportFormats.Cashflow];
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyActualEstimateTypes = actualEstimateTypes.Except(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# Best Estimate initialization

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();